In [1]:
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'numpy'

In [ ]:
def exponential_smoothing(series, alpha, beta, gamma, h, n_preds):
    level, trend = series[0], series[1] - series[0]
    seasonals = initial_seasonality(series, h)
    result = [series[0]]
    for i in range(1, len(series) + n_preds):
        if i >= len(series):
            m = i - len(series) + 1
            result.append(level + m * trend + seasonals[i % h])
        else:
            val = series[i]
            last_level, level = level, alpha * (val - seasonals[i % h]) + (1 - alpha) * (level + trend)
            trend = beta * (level - last_level) + (1 - beta) * trend
            seasonals[i % h] = gamma * (val - level) + (1 - gamma) * seasonals[i % h]
            result.append(level + trend + seasonals[i % h])
    return result

In [ ]:
def initial_seasonality(series, h):
    seasonals = np.zeros(h)
    season_averages = [sum(series[i:i + h]) / h for i in range(len(series) - h + 1)]
    for i in range(h):
        seasonals[i] = sum([series[j] - season_averages[j // h] for j in range(i, len(series) - h + i, h)]) / (len(series) // h)
    return seasonals

In [ ]:
df = pd.read_csv("electrical_load_data.csv")

In [ ]:
alpha = 0.5
beta = 0.5
gamma = 0.5
h = 24
n_preds = 48 

In [ ]:
series = df["load"].values

predictions = exponential_smoothing(series, alpha, beta, gamma, h, n_preds)

In [ ]:
df["prediction"] = np.concatenate((series, predictions))

df.to_csv("electrical_load_predictions.csv", index=False)